# Proof of Concept : communication avec Netbox

Voici deux versions de la communication avec Netbox :
1. une version sans client, uniquement avec des requetes http et des requetes construites à la main
2. (RECOMMANDEE) une version avec le client sgqlc (simple graphQL client) qui prend en charge les requetes http et la constrution de la query à partir de classes Python

Les requêtes exemples cherchent à récupérer des informations sur les box définies dans le netbox de dev (il n'y en à qu'une seule à l'heure de l'écriture)

## Version 1 : sans Client GraphQl

Doc de présentation de grahphql: https://demo.netbox.dev/static/docs/graphql-api/overview/

Importation modules et définition token (Dans Netbox/Admin/Authentification/API Tokens)

In [2]:
import requests
import json #pour avoir un affichage plus lisible

token_netbox = "91a0e80dd5c290ba0d7f7d070a9f3fd80a61e6ea"

headers = {
    'Authorization': 'Token ' + token_netbox,
    'Accept': 'application/json',
}


url = 'http://netbox.dev.fai.rezel.net/graphql/'

Ici on détermine quelles informations demander à Netbox

On peut directement tester la formation d'une requete depuis l'UI web de netbox en cliquant sur GraphQl API en bas de la page, ce qui amène vers une UI (GraphiQL) pour tester les queries)

Il est possible d'ajouter des arguments aux queries (pour ajouter des filtres sur les données qu'on récupère)

In [3]:
query_content = \
'{ \
  device_list(role: "box") { \
    name\
    site {\
      id\
    }\
    rack {\
      id\
    }\
    primary_ip4 {\
      id\
    }\
    primary_ip6 {\
      id\
    }\
  }\
}'

json_data = {
    'query': 'query '+query_content,
}


Execution de la requete et affichage

In [4]:
response = requests.post(url=url, headers=headers, json=json_data)

In [5]:
json_data = response.json()
print(json.dumps(json_data, indent=4))

{
    "data": {
        "device_list": [
            {
                "name": "BOX KLEY-550",
                "site": {
                    "id": "4"
                },
                "rack": {
                    "id": "16"
                },
                "primary_ip4": null,
                "primary_ip6": null
            }
        ]
    }
}


## Version 2 : Avec un GraphQL Client : sgqlc

Importation modules (dont le client [sggqlc](https://github.com/profusion/sgqlc)), définition constantes.
On crée un nouvel objet Endpoint qu'on utilisera pour faire nos requêtes

In [5]:
# sgqlc = simple graphql client
from sgqlc.endpoint.http import HTTPEndpoint
from sgqlc.types import Type, Field, list_of
from sgqlc.operation import Operation
import re #pour transformer les noms de champs de camelCase en snake_case

url = "http://netbox.dev.fai.rezel.net/graphql/"
token_netbox = "91a0e80dd5c290ba0d7f7d070a9f3fd80a61e6ea"
headers = {
    'Authorization': 'Token ' + token_netbox,
    'Accept': 'application/json',
}

def camel_to_snake(string : str):
    """Convertit une chaine de caractère de camelCase en snake_case"""
    pattern = re.compile(r'(?<!^)(?=[A-Z])')
    return pattern.sub('_', string).lower()

endpoint = HTTPEndpoint(url, headers)

On définit avec des classes la structure du Schéma de base de données de Netbox (avantage : typage statique & plus facile à lire et à comprendre)

In [6]:
class IpAddressType(Type):
    id = int
    address = str

class Site(Type):
    id = int
    name = str

class Rack(Type):
    id = int
    name = str

class Device(Type):
    id = int
    name = str
    site = Field(Site)
    rack = Field(Rack)
    primary_ip4 = Field(IpAddressType)
    primary_ip6 = Field(IpAddressType)

class Query(Type):
    device_list = Field(list_of('Device'), args={'role': str})

Contruction de la query (on choisit quels champs requeter)

In [8]:
op = Operation(Query)
devices = op.device_list(role='box')
devices.id()
devices.name()
devices.site()
# devices.rack()  # ici on ne récupère pas les informations sur le rack
devices.primary_ip4()
devices.primary_ip6()

#on est obligé ne snakifier la query car par défaut elle est en camelCase
snakified_query = camel_to_snake(str(op)) 
print(snakified_query)

query {
  device_list(role: "box") {
    id
    name
    site {
      id
      name
    }
    primary_ip4 {
      id
      address
    }
    primary_ip6 {
      id
      address
    }
  }
}


On tape finalement l'API netbox avec notre query puis on l'affiche

In [11]:
data = endpoint(snakified_query)
print(json.dumps(data, indent=4))

{
    "data": {
        "device_list": [
            {
                "id": "116",
                "name": "BOX KLEY-550",
                "site": {
                    "id": "4",
                    "name": "kley"
                },
                "primary_ip4": null,
                "primary_ip6": null
            }
        ]
    }
}
